### Imports

In [1]:
import shapefile
from bs4 import BeautifulSoup
import pixie
import math
import scipy.spatial
import numpy as np
import collections

### Extracting Ireland country data
Data source: https://www.townlands.ie/page/download/

In [2]:
sf = shapefile.Reader("counties/counties.shp")
shapes = sf.shapes()

In [3]:
shape = shapes[0]
print(len(shape.parts))
print(len(shape.points))
print(shape.shapeTypeName)
print(shape.points[0])
print(max([len(shape.parts) for shape in shapes]))

1
8938
POLYGON
(-7.3395627, 54.1467174)
283


In [4]:
polys = []
for shape in shapes:
    parts = shape.parts[:]
    parts.append(len(shape.points))
    for i in range(len(parts)-1):
        polys.append(shape.points[parts[i]:parts[i+1]])

### Extracting triangulation pillar data
Downloaded the data here: http://www.trigpointing-ireland.org.uk/index.php

In [5]:
with open("TPI.gpx.xml", "r") as f:
    xml = BeautifulSoup(f.read())

In [6]:
print(xml.prettify())

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<html>
 <body>
  <gpx creator="GPS TrackMaker" version="1.1" xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd">
   <metadata>
    <link href="http://www.gpstm.com"/>
    <text>
     Geo Studio Tecnology Ltd
    </text>
    <time>
     2009-03-06T20:53:15Z
    </time>
    <bounds maxlat="55.366155" maxlon="-5.467966" minlat="51.441980" minlon="-10.559680">
    </bounds>
   </metadata>
   <wpt lat="54.510601935" lon="-7.485904709">
    <ele>
     158.000000
    </ele>
    <time>
     2009-03-06T00:00:00Z
    </time>
    <name>
     H3362
    </name>
    <cmt>
     Mossfield
    </cmt>
    <desc>
     Mossfield
    </desc>
    <sym>
     City (Small)
    </sym>
   </wpt>
   <wpt lat="54.162722319" lon="-6.488223230">
    <ele>
     290.000000
    </ele>
    <time>
     2009-03-06T00:00:00Z
    

In [7]:
pillarElements = xml.findAll("wpt")
pillars = []
for pillarEle in pillarElements:
    lat, lon = map(lambda name: float(pillarEle.attrs[name]),
                   ["lat", "lon"])
    descEle = pillarEle.find("desc")
    name = descEle.contents[0].strip()
    pillars.append(dict(lat=lat, lon=lon, name=name))

In [8]:
pillars

[{'lat': 54.510601935, 'lon': -7.485904709, 'name': 'Mossfield'},
 {'lat': 54.162722319, 'lon': -6.48822323, 'name': 'Annacloghmullin'},
 {'lat': 54.324645114, 'lon': -7.407539428, 'name': 'Hazelbank'},
 {'lat': 54.767526862, 'lon': -6.501496377, 'name': 'Aughrim Hill'},
 {'lat': 54.323277337, 'lon': -6.397805803, 'name': 'Babylon'},
 {'lat': 54.60795764, 'lon': -5.656996105, 'name': 'Cronstown'},
 {'lat': 54.412210272, 'lon': -6.054377287, 'name': 'Ballykeel Fort'},
 {'lat': 54.641936439, 'lon': -5.801442437, 'name': 'Ballycultra'},
 {'lat': 54.372859192, 'lon': -5.7304316, 'name': 'Craigs Rocks'},
 {'lat': 54.863091438, 'lon': -6.428646949, 'name': 'Tully'},
 {'lat': 54.428489077, 'lon': -7.289867124, 'name': 'Ballyness'},
 {'lat': 54.319279263, 'lon': -6.42854466, 'name': 'Ballysheil'},
 {'lat': 54.379090795, 'lon': -7.70099689, 'name': 'Banagher'},
 {'lat': 54.781843555, 'lon': -6.385012806, 'name': 'Battery Hill'},
 {'lat': 54.294611725, 'lon': -7.654461415, 'name': 'Carney Hill'}

In [9]:
print(len(pillars))

868


### Defining the edges of the map & normalising the coordinates
These calculations should only be run once, otherwise it'll mess up the coordinates and they'll have to be reloaded.

In [10]:
min_lon, max_lon = math.inf, -math.inf
min_lat, max_lat = math.inf, -math.inf

for poly in polys:
    for point in poly:
        lon, lat = point
        min_lon = min(lon, min_lon)
        max_lon = max(lon, max_lon)
        min_lat = min(lat, min_lat)
        max_lat = max(lat, max_lat)

In [11]:
print(min_lon, max_lon, min_lat, max_lat)
print()
print()

-10.6626168 -5.4268157 51.388867 55.4352974




In [12]:
gap = 0.3
min_lon -= gap
max_lon += gap
min_lat -= gap
max_lat += gap

Normalise the boundaries of the country.

In [13]:
for poly in polys:
    for i in range(len(poly)):
        lon, lat = poly[i]
        lon = (lon-min_lon)/(max_lon-min_lon)
        lat = (lat-min_lat)/(max_lat-min_lat)
        poly[i] = (lon, lat)

And normalise the coordinates of the pillars.

In [14]:
for pillar in pillars:
    lon, lat = pillar["lon"], pillar["lat"]
    # Nice copy & paste job.
    lon = (lon-min_lon)/(max_lon-min_lon)
    lat = (lat-min_lat)/(max_lat-min_lat)
    pillar["lon"], pillar["lat"] = lon, lat

### Draw the map

In [15]:
def to_image_coords(W, H, x, y):
    return x*W, (1-y)*H

def draw_line(ctx, W, H, P1, P2):
    # P1 and P2 are assumed to have normalised x & y coordinates
    # in the range [0,1], these are converted to points in the image.
    # Also has to invert the y value since positive y direction is down
    # in drawing interfaces (usually).
    ctx.stroke_segment(*to_image_coords(W, H, *P1),
                       *to_image_coords(W, H, *P2))

def draw_pillar(image, paint, W, H, P):
    path = pixie.Path()
    path.polygon(*to_image_coords(W, H, *P), 3, 3)
    image.fill_path(path, paint)

def draw_ireland(ctx, W, H, polys):
    paint = pixie.Paint(pixie.SOLID_PAINT)
    paint.color = pixie.parse_color("#000000")

    ctx.stroke_style = paint
    ctx.line_width = 1

    for poly in polys:
        for i in range(len(poly)-1):
            draw_line(ctx, W, H, poly[i], poly[i+1])
        draw_line(ctx, W, H, poly[-1], poly[0])

def draw_pillars(ctx, image, W, H, pillars):
    pillar_paint = pixie.Paint(pixie.SOLID_PAINT)
    pillar_paint.color = pixie.Color(0, 0, 1, 1)
    ctx.fill_style = pillar_paint
    for pillar in pillars:
        draw_pillar(image, pillar_paint,
                    W, H, (pillar["lon"], pillar["lat"]))

W, H = 300, 360
image = pixie.Image(W, H)
image.fill(pixie.Color(1, 1, 1, 1))
ctx = image.new_context()

draw_ireland(ctx, W, H, polys)
draw_pillars(ctx, image, W, H, pillars)
for pillar in pillars:
    if pillar["name"] == "Benbulbin":
        pillar_paint = pixie.Paint(pixie.SOLID_PAINT)
        pillar_paint.color = pixie.Color(1, 0, 0, 1)
        ctx.fill_style = pillar_paint
        draw_pillar(image, pillar_paint, W, H, (pillar["lon"], pillar["lat"]))
        break

image.write_file("/tmp/map.png")

### Create Delaunay triangulation

In [16]:
tri_data = np.array(
    [(pillar["lon"], pillar["lat"]) for pillar in pillars])
tri = scipy.spatial.Delaunay(tri_data)

In [17]:
tri.simplices[:5]

array([[796, 799, 821],
       [799, 823, 821],
       [826, 798, 796],
       [864, 841, 119],
       [148, 138, 146]], dtype=int32)

In [18]:
def euclidean_dist(pillar1, pillar2):
    return math.sqrt(
        sum((pillar2[n]-pillar1[n])**2 for n in ["lon", "lat"]))

length_threshold = 0.08
triangles = []
for triangle in tri.simplices:
    bad_triangle = False
    for (i, j) in [(0, 1), (1, 2), (2, 0)]:
        P1 = pillars[triangle[i]]
        P2 = pillars[triangle[j]]
        if euclidean_dist(P1, P2) >= length_threshold:
            bad_triangle = True
    if not bad_triangle:
        # Filtering out triangles that have a side length that
        # is unrealistically large.
        triangles.append(triangle)

# Now confirm that we haven't left out any pillars.
covered_pillars = set()
for triangle in triangles:
    for i in triangle:
        covered_pillars.add(i)
missing_pillars = set(range(len(pillars))) - covered_pillars
print(f"Missing {len(missing_pillars)} pillars:")
for i in missing_pillars:
    print(" ", pillars[i]["name"])

Missing 2 pillars:
  Wolftrap Mountain
  Bannanimma (Carrickaveilty)


### Draw pillar mesh!

In [19]:
def draw_mesh(ctx, W, H, pillars, triangles):
    paint = pixie.Paint(pixie.SOLID_PAINT)
    paint.color = pixie.Color(1, 0, 0, 0.7)

    ctx.stroke_style = paint
    ctx.line_width = 0.5
    for triangle in triangles:
        for (i, j) in [(0, 1), (1, 2), (2, 0)]:
            P1 = pillars[triangle[i]]
            P2 = pillars[triangle[j]]
            draw_line(ctx, W, H,
                      (P1["lon"], P1["lat"]),
                      (P2["lon"], P2["lat"]))

W, H = 300, 360
image = pixie.Image(W, H)
image.fill(pixie.Color(1, 1, 1, 1))
ctx = image.new_context()

#draw_ireland(ctx, W, H, polys)
draw_pillars(ctx, image, W, H, pillars)
draw_mesh(ctx, W, H, pillars, triangles)

image.write_file("/tmp/mesh-only-map.png")

### Use law of sines  to construct mesh

First, pick 2 starting pillars. They must be part of the same triangle.

In [20]:
for i, pillar in enumerate(pillars):
    if pillar["name"] == "Scalp Mountain":
        a_index = i
        break
for i, pillar in enumerate(pillars):
    if pillar["name"] == "Creehennan Hill":
        b_index = i
        break
print(a_index, b_index)
for triangle in triangles:
    if a_index in triangle and b_index in triangle:
        print("Yay, they're in this triangle:", triangle)
        break

763 766
Yay, they're in this triangle: [766 763  87]


Calculate the distance between them, and initialise the data structures we need to propagate the distance through the triangle mesh. The amount of error in that distance measurement can be adjusted.

In [54]:
# This is used to track the distances between pillars, i.e. the edge lengths
# in the triangle mesh.
distance_map = {}
def get_distance_key(p1_id, p2_id):
    return tuple(sorted([p1_id, p2_id]))
def add_dist(distance_map, p1_id, p2_id, dist):
    distance_map[get_distance_key(p1_id, p2_id)] = dist
def get_dist(distance_map, p1_id, p2_id):
    return distance_map[get_distance_key(p1_id, p2_id)]
error = 0.0
# Making the distance smaller so it doesn't become massive.
add_dist(distance_map, a_index, b_index,
         (1-error)*euclidean_dist(pillars[a_index], pillars[b_index]))

visited_pillars = set([a_index, b_index])
triangles_by_pillar = collections.defaultdict(list)
# For the pillars we add, we track which triangle they're part of.
connecting_triangles = {}
for triangle in triangles:
    for pillar in triangle:
        triangles_by_pillar[pillar].append(triangle)
def count_visited(triangle, visited_pillars):
    visit_count = 0
    for p in triangle:
        if p in visited_pillars:
            visit_count += 1
    return visit_count
def fetch_new_candidates(triangles_by_pillar, visited_pillars, pillar,
                         connecting_triangles):
    neighbours = set()
    for triangle in triangles_by_pillar[pillar]:
        for neighbour in triangle:
            if (neighbour not in visited_pillars
                 and count_visited(triangle, visited_pillars) >= 2):
                neighbours.add(neighbour)
                connecting_triangles[neighbour] = triangle
    return neighbours
candidate_pillars = set()
for pillar in visited_pillars:
    candidate_pillars |= fetch_new_candidates(
        triangles_by_pillar, visited_pillars, pillar, connecting_triangles)

In [55]:
print(candidate_pillars)
print(connecting_triangles)

{764, 87}
{87: array([766, 763,  87], dtype=int32), 764: array([763, 766, 764], dtype=int32)}


Now propagate the distance through the triangle mesh. We can only propagate to a pillar that is part of a triangle with 2 already-connected pillars (since that means we have a side length of the triangle and can determine the remaining distances).

In [56]:
def update_distances(p_id, left_id, right_id, pillars, distance_map):
    plr_angle = math.radians(calc_angle(p_id, left_id, right_id, pillars))
    prl_angle = math.radians(calc_angle(p_id, right_id, left_id, pillars))
    lpr_angle = 180 - plr_angle - prl_angle
    d = get_dist(distance_map, left_id, right_id)
    # Law of sines.
    pl_dist = d*math.sin(prl_angle)/math.sin(lpr_angle)
    pr_dist = d*math.sin(plr_angle)/math.sin(lpr_angle)
    add_dist(distance_map, p_id, left_id, pl_dist)
    add_dist(distance_map, p_id, right_id, pr_dist)

def calc_angle(p_1, p_2, p_3, pillars):
    """Calculates the angle formed by p_1->p_2->p_3."""
    pill1, pill2, pill3 = pillars[p_1], pillars[p_2], pillars[p_3]
    line_12 = line_between(pill1, pill2)
    line_23 = line_between(pill2, pill3)
    # I was lazy, stole angle code from here...
    # https://stackoverflow.com/questions/35176451/python-code-to-calculate-angle-between-three-point-using-their-3d-coordinates
    angle = np.arccos(np.dot(line_12, line_23)
                      / (np.linalg.norm(line_12) * np.linalg.norm(line_23)))
    if angle >= 180:
        # Ensures we're calculating the interior angle.
        # Not sure if this is necessary.
        return 360-angle
    return angle

def line_between(pill1, pill2):
    return np.array([pill2["lon"] - pill1["lon"],
                     pill2["lat"] - pill1["lat"]])

while candidate_pillars:
    p_id = candidate_pillars.pop()
    triangle = connecting_triangles[p_id]
    p_index = np.where(triangle == p_id)[0][0]
    left = triangle[(p_index-1)%3]
    right = triangle[(p_index+1)%3]
    # We wanna calculate the side lengths connected to 'p'.
    #       /| left
    #      / |
    #     /  |
    #    /   |
    # p ------ right
    # (Left & right might be switched around).
    # Use the law of sines! Given the internal angles of the triangle and
    # a single side length, the remaining side lengths can be calculated.
    update_distances(p_id, left, right, pillars, distance_map)
    visited_pillars.add(p_id)
    fetch_new_candidates(
        triangles_by_pillar, visited_pillars, p_id, connecting_triangles)

Finally, based on the estimated distances and angles, reconstruct the coordinates of all the pillars.

In [57]:
len(visited_pillars) # TODO figure out why this is wrong.

4